In [2]:
from flask import Flask, request, render_template_string
import numpy as np
import pickle
from threading import Thread

# Load the trained model and scaler
with open('rf_model.pkl', 'rb') as model_file:
    rf_model = pickle.load(model_file)
with open('scaler.pkl', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

# Create the Flask app
app = Flask(__name__)

# HTML template as a string
html_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>Random Forest Prediction</title>
</head>
<body>
    <div>
        <h2>Random Forest Prediction</h2>
        <form action="/predict" method="post">
            <label for="LIMIT_BAL">Amount of the given credit (NT dollar):</label>
            <input type="text" id="LIMIT_BAL" name="LIMIT_BAL"><br><br>
            <label for="EDUCATION">Education (1 = graduate school; 2 = university; 3 = high school; 4 = others):</label>
            <input type="text" id="EDUCATION" name="EDUCATION"><br><br>
            <label for="AGE">Age (year):</label>
            <input type="text" id="AGE" name="AGE"><br><br>
            
            <h3>Repayment Status</h3>
            <p>Enter the repayment status for each month. Where 0 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; ... 8 = payment delay for eight months and above.</p>
            <label for="PAY_0">Repayment status in September, 2005:</label>
            <input type="text" id="PAY_0" name="PAY_0"><br><br>
            <label for="PAY_2">Repayment status in August, 2005:</label>
            <input type="text" id="PAY_2" name="PAY_2"><br><br>
            <label for="PAY_3">Repayment status in July, 2005:</label>
            <input type="text" id="PAY_3" name="PAY_3"><br><br>
            <label for="PAY_4">Repayment status in June, 2005:</label>
            <input type="text" id="PAY_4" name="PAY_4"><br><br>
            <label for="PAY_5">Repayment status in May, 2005:</label>
            <input type="text" id="PAY_5" name="PAY_5"><br><br>
            <label for="PAY_6">Repayment status in April, 2005:</label>
            <input type="text" id="PAY_6" name="PAY_6"><br><br>
            
            <h3>Bill Amounts (NT dollar)</h3>
            <label for="BILL_AMT1">Amount of bill statement in September, 2005:</label>
            <input type="text" id="BILL_AMT1" name="BILL_AMT1"><br><br>
            <label for="BILL_AMT2">Amount of bill statement in August, 2005:</label>
            <input type="text" id="BILL_AMT2" name="BILL_AMT2"><br><br>
            <label for="BILL_AMT3">Amount of bill statement in July, 2005:</label>
            <input type="text" id="BILL_AMT3" name="BILL_AMT3"><br><br>
            <label for="BILL_AMT4">Amount of bill statement in June, 2005:</label>
            <input type="text" id="BILL_AMT4" name="BILL_AMT4"><br><br>
            <label for="BILL_AMT5">Amount of bill statement in May, 2005:</label>
            <input type="text" id="BILL_AMT5" name="BILL_AMT5"><br><br>
            <label for="BILL_AMT6">Amount of bill statement in April, 2005:</label>
            <input type="text" id="BILL_AMT6" name="BILL_AMT6"><br><br>
            
            <h3>Payment Amounts (NT dollar)</h3>
            <label for="PAY_AMT1">Amount paid in September, 2005:</label>
            <input type="text" id="PAY_AMT1" name="PAY_AMT1"><br><br>
            <label for="PAY_AMT2">Amount paid in August, 2005:</label>
            <input type="text" id="PAY_AMT2" name="PAY_AMT2"><br><br>
            <label for="PAY_AMT3">Amount paid in July, 2005:</label>
            <input type="text" id="PAY_AMT3" name="PAY_AMT3"><br><br>
            <label for="PAY_AMT4">Amount paid in June, 2005:</label>
            <input type="text" id="PAY_AMT4" name="PAY_AMT4"><br><br>
            <label for="PAY_AMT5">Amount paid in May, 2005:</label>
            <input type="text" id="PAY_AMT5" name="PAY_AMT5"><br><br>
            <label for="PAY_AMT6">Amount paid in April, 2005:</label>
            <input type="text" id="PAY_AMT6" name="PAY_AMT6"><br><br>
            
            <input type="submit" value="Submit">
        </form>
        <h3>{{ prediction_text }}</h3>
    </div>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(html_template)

@app.route('/predict', methods=['POST'])
def predict():
    feature_dict = {
        "LIMIT_BAL_log": 0,
        "EDUCATION": 1,
        "AGE": 2,
        "PAY_0": 3,
        "PAY_2": 4,
        "PAY_3": 5,
        "PAY_4": 6,
        "PAY_5": 7,
        "PAY_6": 8,
        "BILL_AMT1": 9,
        "BILL_AMT2": 10,
        "BILL_AMT3": 11,
        "BILL_AMT4": 12,
        "BILL_AMT5": 13,
        "BILL_AMT6": 14,
        "PAY_AMT1": 15,
        "PAY_AMT2": 16,
        "PAY_AMT3": 17,
        "PAY_AMT4": 18,
        "PAY_AMT5": 19,
        "PAY_AMT6": 20,
    }
    
    features = [0] * len(feature_dict)
    validation_errors = []

    for key, value in request.form.items():
        if key in feature_dict:
            try:
                if key == "LIMIT_BAL":
                    features[feature_dict["LIMIT_BAL_log"]] = np.log(float(value))
                else:
                    features[feature_dict[key]] = float(value)
            except ValueError:
                validation_errors.append(f"Invalid value for {key}")
    
    # Validate education input
    education = request.form.get("EDUCATION")
    if education not in {"1", "2", "3", "4"}:
        validation_errors.append("Invalid value for EDUCATION. Must be 1, 2, 3, or 4.")
    
    # Validate repayment status inputs
    for i in [0, 2, 3, 4, 5, 6]:
        pay_status = request.form.get(f"PAY_{i}")
        if pay_status not in {"0","1","2", "3", "4", "5", "6", "7", "8"}:
            validation_errors.append(f"Invalid value for PAY_{i}. Must be between 0 and 8.")
    
    # If there are validation errors, return them
    if validation_errors:
        return render_template_string(html_template, prediction_text="; ".join(validation_errors))

    try:
        features = np.array(features).reshape(1, -1)
        scaled_features = scaler.transform(features)
        prediction = rf_model.predict(scaled_features)
        return render_template_string(html_template, prediction_text='Predicted Class: {}'.format(prediction[0]))
    except Exception as e:
        return render_template_string(html_template, prediction_text='Error: {}'.format(e))

# Function to run the app
def run_app():
    app.run(debug=True, use_reloader=False)

# Start the Flask app in a new thread
Thread(target=run_app).start()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jun/2024 16:16:53] "GET / HTTP/1.1" 200 -
C:\Users\katre\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [27/Jun/2024 16:17:37] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2024 16:19:24] "POST /predict HTTP/1.1" 200 -
C:\Users\katre\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [27/Jun/2024 16:22:00] "POST /predict HTTP/1.1" 200 -
C:\Users\katre\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [27/Jun/2024 16:23:53] "POST /predict HTTP/1.1" 200 -
C:\Users\katre\anaconda3\lib\site-packages\sklearn\base.py:420: UserWar